# Example: Eigendecomposition of Stoichiometric Matrices
In this example, we'll explore the eigendecomposition of a standard matrix in chemical systems: the stoichiometric matrix $\mathbf{S}$. The stoichiometric matrix is the digital representation of a chemical reaction system.

> __Learning Objectives:__
> 
> By the end of this example, you should be able to:
>
> * __Compute covariance matrices and eigendecompositions:__ Calculate the covariance matrix of a stoichiometric matrix and compute its eigenvalues and eigenvectors using both direct and iterative methods.
> * __Implement and verify power iteration:__ Apply the power iteration method to find the largest eigenvalue and eigenvector of a symmetric matrix and validate results against built-in linear algebra functions.
> * __Interpret eigenvector components for network analysis:__ Analyze eigenvector coefficients to identify important reactions in metabolic networks using scaling transformations.

Let's get started!
___


## Background: What is a stoichiometric matrix?
Suppose we have a set of chemical (or biochemical) reactions $\mathcal{R}$ involving the chemical species (metabolite) set $\mathcal{M}$. Then, the stoichiometric matrix is a $\mathbf{S}\in\mathbb{R}^{|\mathcal{M}|\times|\mathcal{R}|}$ matrix that holds the stoichiometric coefficients $\sigma_{ij}\in\mathbf{S}$ such that:
* $\sigma_{ij}>0$: Chemical species (metabolite) $i$ is _produced_ by reaction $j$. Species $i$ is a product of reaction $j$.
* $\sigma_{ij} = 0$: Chemical species (metabolite) $i$ is not connected with reaction $j$
* $\sigma_{ij}<0$: Chemical species (metabolite) $i$ is _consumed_ by reaction $j$. Species $i$ is a reactant of reaction $j$.

Thus, the stoichiometric matrix $\mathbf{S}$ encodes the complete connectivity information of the chemical reaction system for, in this case, a biochemical reaction network. Thus, it is the digital representation of the reaction network inside of a cell.
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

  Activating project at `~/Documents/GitHub/CHEME-5820-Lectures-Spring-2026/lectures/week-2/L2a`
   Installed NNlib ─────────── v0.9.33
   Installed KernelFunctions ─ v0.10.67
    Updating `~/Documents/GitHub/CHEME-5820-Lectures-Spring-2026/lectures/week-2/L2a/Project.toml`
  [c3611d14] + ColorVectorSpace v0.11.0
  [5ae59095] + Colors v0.13.1
  [31c24e10] + Distributions v0.25.123
  [5789e2e9] + FileIO v1.17.1
  [cd3eb016] + HTTP v1.10.19
  [033835bb] + JLD2 v0.6.3
  [682c06a0] + JSON v1.4.0
  [ec8451be] + KernelFunctions v0.10.67
  [872c559c] + NNlib v0.9.33
  [91a5bcdd] + Plots v1.41.4
  [10745b16] + Statistics v1.11.1
  [37e2e46d] ~ LinearAlgebra ⇒ v1.12.0
    Updating `~/Documents/GitHub/CHEME-5820-Lectures-Spring-2026/lectures/week-2/L2a/Manifest.toml`
  [79e6a3ab] + Adapt v4.4.0
  [66dad0bd] + AliasTables v1.1.3
  [a9b6321e] + Atomix v1.1.2
  [d1d4a3ce] + BitFlags v0.1.9
  [d360d2e6] + ChainRulesCore v1.26.0
  [0b6fb165] + ChunkCodecCore v1.0.1
  [4c0bbee4] + ChunkCodecLibZlib v1

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Data
We developed a simple software development kit (SDK) against [the BiGG Models application programming interface at the University of California, San Diego](http://bigg.ucsd.edu/). The [BiGG Models database](http://bigg.ucsd.edu/) integrates published genome-scale metabolic networks into a single database with standardized nomenclature and structure. 

> __What are we doing here?__
> 
> We are going to download a stoichiometric matrix from [the BiGG models database](http://bigg.ucsd.edu/) using [the BiGG models API](http://bigg.ucsd.edu/data_access) and then compute its eigendecomposition. 
> * [The BiGG models API](http://bigg.ucsd.edu/data_access) allows users to programmatically access genome-scale stoichiometric model reconstructions using a simple web API. There are `108` models of intracellular biochemistry occurring in various organisms (including humans) in the database (so far); [see here for a list of models](http://bigg.ucsd.edu/models).
> * Here, we'll first explore the [core metabolic model of Palsson and coworkers](https://pubmed.ncbi.nlm.nih.gov/26443778/), which is a scaled-down model of [carbohydrate metabolism](https://en.wikipedia.org/wiki/Carbohydrate_metabolism) in _E.coli_. This model has 72 metabolites and 95 reactions. We'll then look at other models, and see what is going on with these.
> 
>  We call the model download endpoint of [the BiGG models API](http://bigg.ucsd.edu/data_access) and then save the model file to disk (so we don't hit the API unless we have to). 

This call returns model information organized as [a Julia dictionary](https://docs.julialang.org/en/v1/base/collections/#Base.Dict) in the `model::Dict{String, Any}` variable. If a model file is saved, we use the cached file instead of making an API call.

In [2]:
model = let

    # build download endpoint -
    baseurl = "http://bigg.ucsd.edu"; # base url to download model
    modelid = "e_coli_core"; # model id to download (change as needed)
    path_to_saved_model_file = joinpath(_PATH_TO_DATA, "saved-model-$(modelid).jld2");

    # check: do we have a model file saved?
    model = nothing;
    if (isfile(path_to_saved_model_file) == false)
        
        endpoint = MyBiggModelsDownloadModelEndpointModel();
        endpoint.bigg_id = modelid;
        url = build(baseurl, endpoint)
        model = MyBiggModelsDownloadModelEndpointModel(url);

        # Before we move on, save this model for later (so we don't keep hitting the API)
        save(path_to_saved_model_file, Dict("model" => model));
    else
        model = load(path_to_saved_model_file)["model"];
    end
    model; # return the model (either saved, or downloaded)
end

JSON.Object{String, Any} with 6 entries:
  "metabolites"  => Any[Object{String, Any}("id"=>"glc__D_e", "name"=>"D-Glucos…
  "reactions"    => Any[Object{String, Any}("id"=>"PFK", "name"=>"Phosphofructo…
  "genes"        => Any[Object{String, Any}("id"=>"b1241", "name"=>"adhE", "not…
  "id"           => "e_coli_core"
  "compartments" => Object{String, Any}("c"=>"cytosol", "e"=>"extracellular spa…
  "version"      => "1"

__Metabolite records__: Each metabolite (chemical compound) in the network has an associated metabolite record with several fields. Let's take a look at the metabolite at index `1`. The key field for today in the metabolite record is the `id` field, an abbreviation or symbol associated with this metabolite.

In [3]:
model["metabolites"][1] # example metabolite record

JSON.Object{String, Any} with 7 entries:
  "id"          => "glc__D_e"
  "name"        => "D-Glucose"
  "compartment" => "e"
  "charge"      => 0
  "formula"     => "C6H12O6"
  "notes"       => Object{String, Any}("original_bigg_ids"=>Any["glc_D_e"])
  "annotation"  => Object{String, Any}("bigg.metabolite"=>Any["glc__D"], "biocy…

__Reaction records__: Similarly, each reaction in the network has a reaction record with several fields. Let's look at the reaction record at index `25`. The key field for the reaction record is the `metabolites` field, which lists the stoichiometric coefficients associated with this particular reaction.

In [4]:
model["reactions"][25] # example reaction record

JSON.Object{String, Any} with 10 entries:
  "id"                    => "BIOMASS_Ecoli_core_w_GAM"
  "name"                  => "Biomass Objective Function with GAM"
  "metabolites"           => Object{String, Any}("3pg_c"=>-1.496, "accoa_c"=>-3…
  "lower_bound"           => 0.0
  "upper_bound"           => 1000.0
  "gene_reaction_rule"    => ""
  "objective_coefficient" => 1.0
  "subsystem"             => "Biomass and maintenance functions"
  "notes"                 => Object{String, Any}("original_bigg_ids"=>Any["Biom…
  "annotation"            => Object{String, Any}("bigg.reaction"=>Any["BIOMASS_…

In [5]:
model["reactions"][25]["metabolites"]

JSON.Object{String, Any} with 23 entries:
  "3pg_c"    => -1.496
  "accoa_c"  => -3.7478
  "adp_c"    => 59.81
  "akg_c"    => 4.1182
  "atp_c"    => -59.81
  "coa_c"    => 3.7478
  "e4p_c"    => -0.361
  "f6p_c"    => -0.0709
  "g3p_c"    => -0.129
  "g6p_c"    => -0.205
  "gln__L_c" => -0.2557
  "glu__L_c" => -4.9414
  "h2o_c"    => -59.81
  "h_c"      => 59.81
  "nad_c"    => -3.547
  "nadh_c"   => 3.547
  "nadp_c"   => 13.0279
  "nadph_c"  => -13.0279
  "oaa_c"    => -1.7867
  ⋮          => ⋮

__Stoichiometric matrix__: Next, let's build a stoichiometric matrix $\mathbf{S}$ using the metabolite and reaction records. We'll do this using two for loops. 

> __Strategy__: In the outer loop, we iterate over the system's metabolites (chemical species) and select the `id` from the metabolites record for each metabolite. In the inner loop, we iterate over each reaction. For each reaction record, we ask if this reaction has an entry for the current metabolite `id` value; if it does, we grab the stoichiometric coefficient $\sigma_{ij}$ corresponding to this metabolite and reaction.

We'll save the stoichiometric matrix in the `S::Matrix{Float64}` variable.

In [6]:
S = let

    # get some data from the model -
    m = model["metabolites"]; # get list of metabolites
    r = model["reactions"]; # get list of reactions
    number_of_rows = length(m); # how many metabolites do we have? (rows)
    number_of_cols = length(r); # how many reactions do we have? (cols)
    S = zeros(number_of_rows,number_of_cols); # initialize an empty stoichiometric matrix

    # let's build a stm -
    for i ∈ eachindex(m)
        metabolite = m[i]["id"]; # we are checking if this metabolite is in the reaction record
        for j ∈ eachindex(r)
            reaction = r[j];
            if (haskey(reaction["metabolites"], metabolite) == true)
                S[i,j] = reaction["metabolites"][metabolite];
            end
        end
    end
    S; 
end;

In [7]:
S

72×95 Matrix{Float64}:
  0.0  0.0   0.0  0.0   0.0  0.0   0.0  …  0.0  0.0   0.0  0.0  0.0  0.0  0.0
  0.0  0.0   0.0  0.0   0.0  0.0   0.0     0.0  0.0   0.0  0.0  0.0  0.0  0.0
  0.0  0.0   0.0  0.0   0.0  0.0   0.0     0.0  0.0   0.0  0.0  0.0  0.0  0.0
  0.0  0.0   0.0  0.0   0.0  0.0   0.0     0.0  0.0   0.0  0.0  0.0  0.0  0.0
  0.0  0.0   0.0  0.0   0.0  0.0   0.0     0.0  0.0   0.0  0.0  0.0  0.0  0.0
  0.0  0.0   0.0  0.0   0.0  0.0   0.0  …  0.0  0.0   0.0  0.0  0.0  0.0  0.0
  0.0  0.0   0.0  0.0  -1.0  0.0   0.0     0.0  0.0   0.0  0.0  0.0  0.0  0.0
  0.0  0.0   0.0  0.0   0.0  0.0   0.0     0.0  0.0   0.0  0.0  0.0  0.0  0.0
  1.0  0.0   0.0  0.0   1.0  1.0   1.0     0.0  0.0  -4.0  0.0  0.0  0.0  0.0
  0.0  0.0   0.0  0.0   0.0  0.0  -1.0     0.0  0.0   3.0  0.0  0.0  0.0  0.0
  ⋮                          ⋮          ⋱             ⋮                   
 -1.0  0.0   1.0  0.0   0.0  0.0   0.0     0.0  0.0   0.0  0.0  0.0  0.0  0.0
  1.0  0.0   0.0  0.0   0.0  0.0   0.0     0

___

## Task 1: Compute the covariance matrix
In this task, we'll compute the covariance matrix $\mathbf{\Sigma}$ of the stoichiometric matrix $\mathbf{S}$.

> __Why are we doing this?__ The stoichiometric matrix is not square. Thus, we cannot directly compute its eigendecomposition. However, suppose we compute [the covariance matrix](https://en.wikipedia.org/wiki/Covariance_matrix) between the columns, i.e., between the reactions stoichiometric vectors. This gives us an idea about the relationship between the network's reactions $i$ and $j$.

Let's start by computing the covariance matrix $\mathbf{\Sigma}$ using [the `cov(...)` function](https://docs.julialang.org/en/v1/stdlib/Statistics/#Statistics.cov) from the `Statistics.jl` package (this will be our ground truth). We'll save this covariance matrix in the `Σ::Matrix{Float64}` variable.

In [8]:
Σ = cov(S) # this is col covariance matrix from Statistics.jl: Ground truth!

95×95 Matrix{Float64}:
  0.0702269     0.0        -0.0140845  …  0.0       0.0        0.0
  0.0           0.056338    0.0           0.0       0.0        0.0422535
 -0.0140845     0.0         0.028169      0.0       0.0        0.0
  0.028169      0.0         0.0           0.0       0.0        0.0
  0.0140845     0.0         0.0           0.0       0.0        0.0
  0.0140845     0.028169    0.0        …  0.0       0.0        0.056338
  0.0140845     0.0         0.0           0.0       0.0        0.0
  0.0           0.0         0.0           0.0       0.0        0.0
  0.0140845     0.0         0.0           0.0       0.0        0.0
  0.0138889     0.0         0.0           0.0       0.0        0.028169
  ⋮                                    ⋱                      
  0.028169      0.0         0.0           0.0       0.0        0.0
  0.0138889     0.0         0.0           0.0       0.0        0.028169
 -0.000195618  -0.0140845   0.0           0.0       0.0        0.028169
 -0.000195618  -0

Next, let's compute the empirical covariance matrix $\hat{\mathbf{\Sigma}}\in\mathbb{R}^{|\mathcal{R}|\times|\mathcal{R}|}$. The empirical covariance matrix is computed as:
$$
\hat{\mathbf{\Sigma}} = \frac{1}{n-1}\tilde{\mathbf{X}}^{\top}\tilde{\mathbf{X}}
$$
where $\tilde{\mathbf{X}}\in\mathbb{R}^{n\times{|\mathcal{R}|}}$ is the mean-centered stoichiometric matrix (i.e., each column has zero mean), and $n$ is the number of metabolites (rows) in the stoichiometric matrix. We'll verify our implementation by comparing our matrix with the one computed using [the `cov(...)` function](https://docs.julialang.org/en/v1/stdlib/Statistics/#Statistics.cov) from the `Statistics.jl` package.

Let's save the covariance matrix in the `Σ̂::Matrix{Float64}` variable.

In [9]:
Σ̂ = let

    # initialize -
    number_of_metabolites = size(S,1); # rows are metabolites
    number_of_reactions = size(S,2); # cols are reactions
    m = mean(S, dims=1) |> vec; # mean of each column
    ones_vector = ones(number_of_metabolites); # ones vector
    B = ones_vector * transpose(m); # outer product to center data

    X̂ = S - B; # centered data matrix
    Σ̂ = (transpose(X̂) * X̂) / (number_of_metabolites - 1); # compute covariance matrix
    Σ̂; # return covariance matrix
end

95×95 Matrix{Float64}:
  0.0702269     0.0        -0.0140845  …  0.0       0.0        0.0
  0.0           0.056338    0.0           0.0       0.0        0.0422535
 -0.0140845     0.0         0.028169      0.0       0.0        0.0
  0.028169      0.0         0.0           0.0       0.0        0.0
  0.0140845     0.0         0.0           0.0       0.0        0.0
  0.0140845     0.028169    0.0        …  0.0       0.0        0.056338
  0.0140845     0.0         0.0           0.0       0.0        0.0
  0.0           0.0         0.0           0.0       0.0        0.0
  0.0140845     0.0         0.0           0.0       0.0        0.0
  0.0138889     0.0         0.0           0.0       0.0        0.028169
  ⋮                                    ⋱                      
  0.028169      0.0         0.0           0.0       0.0        0.0
  0.0138889     0.0         0.0           0.0       0.0        0.028169
 -0.000195618  -0.0140845   0.0           0.0       0.0        0.028169
 -0.000195618  -0

### Check: Covariance matricies the same?
Let's check that our computed covariance matrix `Σ̂` is the same as the one computed using [the `cov(...)` function](https://docs.julialang.org/en/v1/stdlib/Statistics/#Statistics.cov) from the `Statistics.jl` package. We'll use [the `isapprox(...)` function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to check if the two matrices are approximately equal.

In [10]:
let
    # initialize -
    atol = 1e-8; # absolute tolerance for comparison
    is_equal = isapprox(Σ, Σ̂; atol = atol);
    @assert is_equal == true "Covariance matrices are not approximately equal!";
end

__Did we blow up?__ If not, great! Our covariance matrix implementation is correct. Let's move on and compute the eigendecomposition of the emphirical covariance matrix.

___

## Task 2: Compute the largest eigenvalue/eigenvector
In this task, we'll compute the largest eigenvalue and associated eigenvector of the $\mathbf{\Sigma}$ matrix using the [power iteration method](https://en.wikipedia.org/wiki/Power_iteration). 

However, before we do that, let's use Julia's built-in [eigendecomposition function `eigen(...)`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) to compute all the eigenvalues and eigenvectors of the $\mathbf{\Sigma}$ matrix (so we have something to compare against).

> __Why compute all the values__? We'll use [the `eigen(...)` method](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen) and then grab the last value(s) when we look at the largest value, this method returns all the eigenvalues and eigenvectors, sorted in ascending order. Thus, we only really need the last value for this example, but later we'll use the rest. 

We'll store the eigenvalues in the diagonal $\mathbf{\Lambda}$ matrix, while the eigenvectors will be stored in the $\mathbf{V}$ matrix.

In [11]:
Λ,V = let

    # initialize -
    A = Σ̂; # this is the matrix that we will decompose (if we are looking Σ̂, then change)
    (n,m) = size(A); # what is the dimension of A?
    Λ = Matrix{Float64}(1.0*I, n, n); # builds the I matrix, we'll update with λ -
    
    # Decompose using the built-in function
    F = eigen(A);   # eigenvalues and vectors in F of type Eigen
    λ = F.values;   # vector of eigenvalues
    V = F.vectors;  # n x n matrix of eigenvectors, each col is an eigenvector

    # package the eigenvalues into Λ -
    for i ∈ 1:n
        Λ[i,i] = λ[i];
    end

    Λ,V
end;

In [12]:
Λ[end,end] # largest eigenvalue

258.38801665121497

Now let's compute the largest eigenvalue and eigenvector using the power iteration method.

> __What is going on here?__ The power iteration method iteratively computes the dominant eigenpair by multiplying a guess vector by matrix $\mathbf{A}$ and normalizing. We specify the matrix `A::Matrix{Float64}`, initial vector `v::Vector{Float64}`, maximum iterations `maxiter::Int`, and convergence tolerance `ϵ::Float64`.

We'll save the largest eigenvalue in the `λ₁::Float64` variable and the associated eigenvector in the `v₁::Vector{Float64}` variable.

In [13]:
λ₁,v₁ = let

    # initialize -
    A = Σ̂; # this is the matrix that we will decompose (free to change)
    (n,m) = size(A); # what is the dimension of A?
    v = ones(n); # initial guess for the eigenvector
    maxiter = 1000; # maxinum number of iterations
    ϵ = 0.000000001; # tolerance

    # call poweriteration method
    result = poweriteration(A,v, maxiter = maxiter, ϵ = ϵ);

    # pull data from result 
    (result.value, result.vector)
end;

Converged in 4 iterations


In [14]:
λ₁ # largest eigenvalue estimate from power iteration

258.3880166511564

In [15]:
V[:,end] # largest eigenvector from built-in function

95-element Vector{Float64}:
  0.009763763213526728
 -0.0002528754258268639
  7.224088364230286e-6
  0.006607585840270487
  0.006535921381242149
  0.0032528674240287526
  0.0035043554202608654
 -8.199419437096758e-5
  0.006541327254849948
  0.003623122163380241
  ⋮
  0.006559948356308675
  0.0035249934719769616
  0.0001941433826520636
 -0.0016123581121367137
 -0.013461659108511374
  0.0018065014947887814
 -2.541445801584885e-7
  2.7031166194291245e-7
  0.00013550653860427647

To test our power iteration implementation, let's compare the values of the largest eigenvalue, eigenvector pair $(\lambda_{1}, \mathbf{v}_{1})$ that we just estimated with those computed [using the `eigen(...)` function exported by the LinearAlgebra.jl package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigen). 

> __Check__: We use the [`@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the shortcut version of [the `isapprox(...)` function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare our result to the built in function. If the argument to [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) evaluates to `false`, an [`AssertionError` instance](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is thrown. We'll compare the values to within an absolute tolerance value of `atol::Float64`. The tolerance `atol = 1e-4` is sufficiently tight to ensure agreement while accounting for accumulated floating-point error in iterative computation.

So does our power iteration method work?

In [16]:
let
    
    # initialize -
    λ = Λ[end,end]; # largest eigenvalue is in the last row/col of Λ
    v = V[:,end]; # largest eigenvector is in the last column of V
    atol = 1e-4; # tolerance for comparison

    # tests
    @assert isapprox(λ, λ₁, atol = atol)  # do the eigenvalues and eigenvectors match?
    @assert isapprox(abs.(v), abs.(v₁), atol = atol); # do the eigenvectors match?
end

If we get here without an error, our power iteration method works! Now that we have confidence in our implementation, let's explore what the largest eigenvalue/eigenvector pair tells us about the stoichiometric matrix $\mathbf{S}$ and the reaction network it represents.

___

## Task 3: Interpret the dominant eigenpair
Now that we've confirmed our power iteration method works, let's use these validated results to extract insights about the reaction network.

> __What can we learn from the eigenvector?__ The eigenvector components encode information about the reactions. Let's explore what different scaling transformations reveal about their relative importance.

First, let's grab the absolute largest component, and see what reaction it corresponds to.
> __Largest absolute component__ 
> 
> The largest component of the largest eigenvector $\mathbf{v}_{1}$ tells us which reaction is (loosely) the most important in the network. However, the components of the eigenvector can be positive or negative, so we need to be a little careful here, and just look at the absolute values of the components. 
> 
> For some eigenvector $\mathbf{z}$, the ith scaled component $\sigma(\mathbf{z})_{i}$ is given by:
> $$
\begin{align*}
    \sigma(\mathbf{z})_{i} &= \frac{\text{abs}(z_{i})}{\sum_{j=1}^{m}\text{abs}(z_{j})}\quad{i=1,2,\dots,m}
\end{align*}    
$$
> The scaled vectors should sum to `1`; thus, we can think about the elements (loosely) as probabilities, i.e., the probability that the ith component is the most important. 

Let's do the scaling, grab the index of the maximum scaled element [using the `argmax(...)` method](https://docs.julialang.org/en/v1/base/collections/#Base.argmax). We'll then pipe `|>` that index into reactions data, and pull out the most important reaction record:

In [17]:
sum(abs.(v₁)) |> T -> (1/T)*abs.(v₁) |> argmax |> i-> model["reactions"][i]

JSON.Object{String, Any} with 10 entries:
  "id"                    => "BIOMASS_Ecoli_core_w_GAM"
  "name"                  => "Biomass Objective Function with GAM"
  "metabolites"           => Object{String, Any}("3pg_c"=>-1.496, "accoa_c"=>-3…
  "lower_bound"           => 0.0
  "upper_bound"           => 1000.0
  "gene_reaction_rule"    => ""
  "objective_coefficient" => 1.0
  "subsystem"             => "Biomass and maintenance functions"
  "notes"                 => Object{String, Any}("original_bigg_ids"=>Any["Biom…
  "annotation"            => Object{String, Any}("bigg.reaction"=>Any["BIOMASS_…

Next, let's look at the top `k` reactions in the network. The following code identifies the reactions with the largest absolute eigenvector components after normalization. By sorting the normalized values, we can identify which reactions contribute most to the dominant eigenvector, providing insights into the network's structure.

> __What is going on here?__ We normalize the absolute eigenvector values to create importance weights, then use [sortperm(...)](https://docs.julialang.org/en/v1/base/sort/#Base.sortperm) to identify the top `k::Int64` reactions ranked by contribution.

Let's save the top `k::Int64` reactions in the `top_k_reactions::Vector{JSON.Object{String, Any}}` array.

In [18]:
top_k_reactions = let

    # initialize -
    k = 10; # how many top reactions do we want?
    normalized_v₁ = sum(abs.(v₁)) |> T -> (1/T)*abs.(v₁); # normalize the eigenvector
    sorted_indices = sortperm(normalized_v₁, rev = true); # get sorted indices (largest to smallest)
    top_k_indices = sorted_indices[1:k]; # get top k indices
    top_k_reactions = [model["reactions"][i] for i ∈ top_k_indices]; # get top k reactions

    # what is the cumulative contribution of these top k reactions?
    cumulative_contribution = sum(normalized_v₁[top_k_indices]);
    println("Cumulative contribution of top $k reactions: $(round(cumulative_contribution*100, digits=2))%");

    top_k_reactions # return top k reactions
end

Cumulative contribution of top 10 reactions: 86.03%


10-element Vector{JSON.Object{String, Any}}:
 JSON.Object("id" => "BIOMASS_Ecoli_core_w_GAM", "name" => "Biomass Objective Function with GAM", "metabolites" => JSON.Object{String, Any}("3pg_c" => -1.496, "accoa_c" => -3.7478, "adp_c" => 59.81, "akg_c" => 4.1182, "atp_c" => -59.81, "coa_c" => 3.7478, "e4p_c" => -0.361, "f6p_c" => -0.0709, "g3p_c" => -0.129, "g6p_c" => -0.205…), "lower_bound" => 0.0, "upper_bound" => 1000.0, "gene_reaction_rule" => "", "objective_coefficient" => 1.0, "subsystem" => "Biomass and maintenance functions", "notes" => JSON.Object{String, Any}("original_bigg_ids" => Any["Biomass_Ecoli_core_w_GAM"]), "annotation" => JSON.Object{String, Any}("bigg.reaction" => Any["BIOMASS_Ecoli_core_w_GAM"], "metanetx.reaction" => Any["MNXR96280"], "sbo" => "SBO:0000629")…)
 JSON.Object("id" => "PPS", "name" => "Phosphoenolpyruvate synthase", "metabolites" => JSON.Object{String, Any}("amp_c" => 1.0, "atp_c" => -1.0, "h2o_c" => -1.0, "h_c" => 2.0, "pep_c" => 1.0, "pi_c" => 1.0, "

Alternatively, let's use [the `softmax(...)` function](src/Compute.jl) to transform the largest eigenvector in a probability vector (sums to one, all entries are non-negative). The [softmax](https://en.wikipedia.org/wiki/Softmax_function) for some vector $\mathbf{z}$ as
$$
\begin{equation}
\sigma(\mathbf{z})_{i} = \frac{e^{z_{i}}}{\sum_{j=1}^{m}e^{z_{j}}}\quad{i=1,2,\dots,m}
\end{equation}
$$
where $\sigma(\mathbf{z})_{i}$ is the ith components of the transformed eigenvector. We apply [the `argmax(...)` function](https://docs.julialang.org/en/v1/base/collections/#Base.argmax) to the transformed vector to get the largest single component, or we could grab the top `k` components as before.

In [18]:
top_k_reactions_softmax = let

    # initialize -
    k = 10; # how many top reactions do we want?
    transformed_v₁ = softmax(v₁); # apply softmax to the eigenvector
    sorted_indices = sortperm(transformed_v₁, rev = true); # get sorted indices (largest to smallest)
    top_k_indices = sorted_indices[1:k]; # get top k indices
    top_k_reactions_softmax = [model["reactions"][i] for i ∈ top_k_indices]; # get top k reactions

    # what is the cumulative contribution of these top k reactions?
    cumulative_contribution = sum(transformed_v₁[top_k_indices]);
    println("Cumulative contribution of top $k reactions (softmax): $(round(cumulative_contribution*100, digits=2))%");

    top_k_reactions_softmax # return top k reactions
end

Cumulative contribution of top 10 reactions (softmax): 12.2%


10-element Vector{JSON.Object{String, Any}}:
 JSON.Object("id" => "BIOMASS_Ecoli_core_w_GAM", "name" => "Biomass Objective Function with GAM", "metabolites" => JSON.Object{String, Any}("3pg_c" => -1.496, "accoa_c" => -3.7478, "adp_c" => 59.81, "akg_c" => 4.1182, "atp_c" => -59.81, "coa_c" => 3.7478, "e4p_c" => -0.361, "f6p_c" => -0.0709, "g3p_c" => -0.129, "g6p_c" => -0.205…), "lower_bound" => 0.0, "upper_bound" => 1000.0, "gene_reaction_rule" => "", "objective_coefficient" => 1.0, "subsystem" => "Biomass and maintenance functions", "notes" => JSON.Object{String, Any}("original_bigg_ids" => Any["Biomass_Ecoli_core_w_GAM"]), "annotation" => JSON.Object{String, Any}("bigg.reaction" => Any["BIOMASS_Ecoli_core_w_GAM"], "metanetx.reaction" => Any["MNXR96280"], "sbo" => "SBO:0000629")…)
 JSON.Object("id" => "PPS", "name" => "Phosphoenolpyruvate synthase", "metabolites" => JSON.Object{String, Any}("amp_c" => 1.0, "atp_c" => -1.0, "h2o_c" => -1.0, "h_c" => 2.0, "pep_c" => 1.0, "pi_c" => 1.0, "

> __Transformation perspectives:__ The normalized approach distributes importance broadly across all eigenvector components, while the softmax transformation exponentially amplifies larger components to emphasize dominant reactions. The lower cumulative contribution from softmax reflects this difference: we can identify either the most critical reactions (softmax) or a more balanced view of reaction roles (normalization).

___

## Summary
The power iteration method efficiently computes the dominant eigenvalue and eigenvector of a covariance matrix, revealing which reactions contribute most to the structure of a metabolic network.

> __Key Takeaways__:
> 
> * **Covariance matrices enable eigendecomposition of non-square matrices:** By computing the covariance matrix of the stoichiometric matrix columns, we transform a non-square matrix into a square, symmetric matrix suitable for eigendecomposition.
> * **Power iteration converges to the dominant eigenpair:** The iterative approach provides a computationally efficient alternative to direct methods, particularly useful for large sparse matrices encountered in genome-scale metabolic networks.
> * **Eigenvector components identify important network reactions:** Scaling eigenvector components reveals which reactions have the strongest influence on network behavior, providing biological insights into metabolic pathway organization.

Eigenvector analysis of metabolic networks reveals the underlying structure that drives biochemical transformations in living systems.
___